# Análisis de Sentimientos con NLTK VADER
## Análisis de reseñas de productos de Amazon

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WillianReinaG/PNL_unidad1/blob/main/2_Analisis_Sentimientos_VADER.ipynb)

Este notebook realiza un análisis completo de sentimientos utilizando NLTK con VADER (Valence Aware Dictionary and sEntiment Reasoner).

**Instalación de dependencias:** Instala las librerías necesarias para análisis de sentimientos, métricas de evaluación y visualización.

In [ ]:
!pip install nltk scikit-learn matplotlib seaborn pandas numpy -q

import nltk
nltk.download("vader_lexicon", quiet=True)
nltk.download("punkt", quiet=True)

print("✓ Dependencias instaladas")

**Importar librerías:** Cargamos SentimentIntensityAnalyzer (VADER), métricas de evaluación, y herramientas de visualización.

In [ ]:
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

sia = SentimentIntensityAnalyzer()
print("✓ Librerías listas")

**Cargar datos:** Lee el archivo con ~1,000 reseñas de Amazon. Cada línea tiene: [texto de reseña TAB etiqueta (0=negativo, 1=positivo)]

In [ ]:
# Detectar si estamos en Google Colab
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Si estamos en Colab, descargar el archivo
if IN_COLAB:
    !wget https://raw.githubusercontent.com/WillianReinaG/PNL_unidad1/main/amazon_cells_labelled.txt -O ./amazon_cells_labelled.txt

# Cargar y procesar el archivo
with open('./amazon_cells_labelled.txt', 'r', encoding='utf-8') as file:
    df = pd.read_csv(file, sep='\t', header=None, names=['review', 'sentiment'])

print(f"Datos cargados: {len(df)} reseñas")
print(f"\nPrimeras 5 reseñas:")
print(df.head())
print(f"\nDistribución de sentimientos:")
print(df['sentiment'].value_counts())
print(f"\nInformación del dataset:")
print(df.info())

**Limpieza de datos:** Normaliza el texto: convierte a minúsculas, elimina URLs, caracteres especiales y espacios múltiples.

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\\S+|www\\S+", "", text)
    text = re.sub(r"[^a-zA-Z0-9\\s.!?,-]", "", text)
    text = re.sub(r"\\s+", " ", text).strip()
    return text

df["review_cleaned"] = df["review"].apply(clean_text)
df = df[df["review_cleaned"].str.len() > 0]
print(f"Reseñas después de limpieza: {len(df)}")

**Análisis VADER:** Aplica VADER a cada reseña. Genera 4 scores: positivo, negativo, neutral y compound (el más importante: -1 a +1).

In [ ]:
sentiment_scores = df["review_cleaned"].apply(lambda x: sia.polarity_scores(x))
sentiment_df = pd.DataFrame(sentiment_scores.tolist())
df = pd.concat([df, sentiment_df], axis=1)
df = df.rename(columns={"neg": "vader_negative", "neu": "vader_neutral", "pos": "vader_positive", "compound": "vader_compound"})

print("✓ Análisis completado")
print(f"Compound score: min={df[\"vader_compound\"].min():.3f}, max={df[\"vader_compound\"].max():.3f}, medio={df[\"vader_compound\"].mean():.3f}")

**Conversión a predicciones:** Convierte los scores continuos (-1 a +1) en predicciones binarias (0 o 1). Threshold: 0.05

In [ ]:
df["vader_prediction"] = df["vader_compound"].apply(lambda x: 1 if x >= 0.05 else 0)

print("Distribución de predicciones:")
print(df["vader_prediction"].value_counts())
print(f"\nPositivas predichas: {(df[\"vader_prediction\"] == 1).sum() / len(df) * 100:.1f}%")

**Métricas de evaluación:** Calcula Accuracy, Precision, Recall y F1-Score comparando predicciones con valores reales.

In [ ]:
y_true = df["sentiment"].values
y_pred = df["vader_prediction"].values

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

print("=" * 60)
print("MÉTRICAS DE CALIDAD")
print("=" * 60)
print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")

**Matriz de confusión:** Muestra verdaderos positivos, falsos negativos, falsos positivos y verdaderos negativos.

In [ ]:
cm = confusion_matrix(y_true, y_pred)

print("\nMatriz de Confusión:")
print(f"{\"\":<20} Predicción Neg  Predicción Pos")
print(f"Real Negativo:      {cm[0,0]:>6}           {cm[0,1]:>6}")
print(f"Real Positivo:      {cm[1,0]:>6}           {cm[1,1]:>6}")

print("\nReporte detallado:")
print(classification_report(y_true, y_pred, target_names=["Negativo", "Positivo"]))

**Visualización 1:** Gráfico de calor de la matriz de confusión.

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Negativo", "Positivo"], yticklabels=["Negativo", "Positivo"])
plt.title("Matriz de Confusión")
plt.ylabel("Real")
plt.xlabel("Predicción")
plt.tight_layout()
plt.show()

**Visualización 2:** Histograma de distribución de scores y box plot por sentimiento real.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].hist(df["vader_compound"], bins=50, edgecolor="black", alpha=0.7)
axes[0].axvline(x=0.05, color="green", linestyle="--", label="Threshold")
axes[0].set_xlabel("Compound Score")
axes[0].set_title("Distribución de Scores")
axes[0].legend()
df.boxplot(column="vader_compound", by="sentiment", ax=axes[1])
axes[1].set_xlabel("Sentimiento Real")
axes[1].set_title("Compound Score por Sentimiento")
plt.suptitle("")
plt.tight_layout()
plt.show()

**Palabras influyentes:** Identifica las 10 palabras más frecuentes en reseñas positivas y negativas.

In [ ]:
positive_reviews = df[df["sentiment"] == 1]["review_cleaned"].str.cat(sep=" ")
negative_reviews = df[df["sentiment"] == 0]["review_cleaned"].str.cat(sep=" ")
positive_words = word_tokenize(positive_reviews)
negative_words = word_tokenize(negative_reviews)
positive_freq = Counter([w for w in positive_words if len(w) > 2 and w.isalpha()])
negative_freq = Counter([w for w in negative_words if len(w) > 2 and w.isalpha()])

print("Palabras en reseñas POSITIVAS:")
for word, freq in positive_freq.most_common(10):
    print(f"  {word:<15} - {freq:>4} veces")
print("\nPalabras en reseñas NEGATIVAS:")
for word, freq in negative_freq.most_common(10):
    print(f"  {word:<15} - {freq:>4} veces")

**Casos mal clasificados:** Analiza falsos positivos y falsos negativos para entender las limitaciones del modelo.

In [ ]:
df["correct"] = df["sentiment"] == df["vader_prediction"]
misclassified = df[~df["correct"]]
correctly_classified = df[df["correct"]]
fp = misclassified[(misclassified["sentiment"] == 0) & (misclassified["vader_prediction"] == 1)]
fn = misclassified[(misclassified["sentiment"] == 1) & (misclassified["vader_prediction"] == 0)]

print("=" * 70)
print("ANÁLISIS DE ERRORES")
print("=" * 70)
print(f"Correctas: {len(correctly_classified)} ({len(correctly_classified)/len(df)*100:.2f}%)")
print(f"Incorrectas: {len(misclassified)} ({len(misclassified)/len(df)*100:.2f}%)")
print(f"  - Falsos Positivos: {len(fp)}")
print(f"  - Falsos Negativos: {len(fn)}")

print(f"\nEjemplos de FALSOS POSITIVOS:")
for idx in fp.head(2).index:
    print(f"  {df.loc[idx, \"review\"][:60]}... | Score: {df.loc[idx, \"vader_compound\"]:.3f}")

print(f"\nEjemplos de FALSOS NEGATIVOS:")
for idx in fn.head(2).index:
    print(f"  {df.loc[idx, \"review\"][:60]}... | Score: {df.loc[idx, \"vader_compound\"]:.3f}")